In [ ]:
from urllib.error import HTTPError, URLError
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import html_to_json
import os
import json

In [ ]:
def scrap(link, category, tries):
    # Save page
    key = '1783319260d70f84da21868ce0fd6207'
    api_link = f"http://api.scraperapi.com?api_key={key}&url={link}"
    if 'https://' in link:
        link.replace('https://', '')
    while True:
        try:
            page=urlopen(api_link)
            return page
        except HTTPError:
            print('\tSleeping...')
            time.sleep(15)
            tries+=1
            print(f'Scrapping: {category}, Try: {tries}')
            scrap(link, category, tries)
            pass

In [ ]:
def write(page):
    # Read data on page
    html_bytes=page.read()
    # Decode data from page
    html=html_bytes.decode("utf-8")
    # Write html file from data scrapped
    with open(f"html_dir/{category}.html","w",encoding="utf-8")as html_file:
        html_file.write(html)

In [ ]:
def read(category):    
    # Read html file from scrapped data
    
    with open(f"html_dir/{category}.html","r",encoding='utf-8')as html_file:
        output=html_file.read()
        
        soup = BeautifulSoup(output, 'lxml')
        if category == 'skjermer':
            products = soup.find_all('div', class_ = 'product-list-item subscription-price-visible')
        else:
            products = soup.find_all('div', class_ = 'product-list-item')
            
        product_dict = {}
        for product in products:
            link = str('komplett.no' + product.a['href'])
            image = str('komplett.no/img/p/800/' + product.a['href'].split('/')[2])
            sale = None
            price_now = product.find('span', class_= 'product-price-now').text
            price_now = int(''.join(char for char in price_now if char.isalnum()))
            price_before = product.find('div', class_= 'product-price-before')
            if price_before != None:
                price_before = product.find('div', class_= 'product-price-before').text.replace('Før', '').replace(',-', '').strip()
                price_before = int(''.join(char for char in price_before if char.isalnum()))
                sale = True
            
            name = product.h2.text.replace('å', 'aa').replace('ø', 'o')
            stats = product.p.text.replace(',','').split()
            available = product.find('span', class_='stockstatus-stock-details').text.replace('å', 'aa').replace('ø', 'o')
            itemnumber = product.find('div', class_='product-data').text.replace(' ', '').replace('\n', '').split('/')[0].split(':')[1]

            product_dict[itemnumber] = [name, price_now, price_before, sale, stats, available, image, link, 'komplett']       
        
        return product_dict

In [ ]:

links = ['https://www.komplett.no/category/11157/tv-lyd-bilde/tv-video/tv-er?nlevel=10719%C2%A730000%C2%A711157&hits=240',
        'www.komplett.no/category/11158/datautstyr/skjermer/skjermer?nlevel=10000%C2%A710392%C2%A711158&hits=264',
        'www.komplett.no/category/21635/gaming/gaming-utstyr/gaming-tastatur?nlevel=10431%C2%A721603%C2%A721635&hits=168',
        'www.komplett.no/category/199889/hjem-fritid/stoevsugere-rengjoering?nlevel=10560%C2%A7199889&hits=120']
single_link = [links[1]]

for link in links:
    tries = 0
    
    # Category on the item in our current link
    category = link.split('?')[0].split('/')[-1].replace('-','_')
    print(category)
    # Write down the data we get from scrapping
    write(scrap(link, category, tries))
    current = read(category)
    # Used for columns in excel
    col = ['name', 'price_now', 'price_before', 'sale', 'stats', 'available', 'image', 'link', 'site']
    # Create dataframe
    df = pd.DataFrame(current.values(), columns = col)
    # Convert dataframe to excel file
    df.to_csv(f'excel_dir/{category}.csv')
    print(f'scrapped: {category}')